# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import json

# Import API key
from api_keys import geoapify_key


In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,32.02,80,75,12.66,GL,1686142420
1,1,nabire,-3.3667,135.4833,77.95,90,100,2.62,ID,1686142420
2,2,remire-montjoly,4.9167,-52.2667,82.44,78,40,3.44,GF,1686142420
3,3,walvis bay,-22.9575,14.5053,67.15,55,0,8.05,NaN,1686142155
4,4,avarua,-21.2078,-159.7750,73.45,73,20,5.75,CK,1686142283
...,...,...,...,...,...,...,...,...,...,...
582,582,damavand,35.7178,52.0650,74.98,33,100,11.65,IR,1686142475
583,583,katsuren-haebaru,26.3369,127.8719,79.57,98,0,9.22,JP,1686142475
584,584,suva,-18.1416,178.4415,69.28,100,75,3.44,FJ,1686142475
585,585,nzerekore,7.7562,-8.8179,83.17,64,100,4.76,GN,1686142475


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [112]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
    
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
#My ideal is no hotter than 75 and humidity of no more than 65%
ideal = pd.DataFrame(city_data_df.loc[(city_data_df["Max Temp"] <= 80) & (city_data_df["Humidity"] <= 75)])

# Drop any rows with null values
ideal_no_null = ideal.dropna()

# Display sample data
ideal_no_null

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,avarua,-21.2078,-159.7750,73.45,73,20,5.75,CK,1686142283
5,5,port alfred,-33.5906,26.8910,76.17,38,0,3.00,ZA,1686142421
9,9,amos,48.5669,-78.1162,53.76,45,56,13.11,CA,1686142421
12,12,bredasdorp,-34.5322,20.0403,62.33,49,47,4.74,ZA,1686142421
15,15,ballina,-28.8667,153.5667,55.78,70,96,0.38,AU,1686142421
...,...,...,...,...,...,...,...,...,...,...
577,577,polyarnyy,69.1989,33.4478,45.70,55,100,6.58,RU,1686142474
578,578,gargalianoi,37.0667,21.6333,75.81,74,100,4.97,GR,1686142474
579,579,lyantor,61.6195,72.1555,72.32,34,60,11.61,RU,1686142475
582,582,damavand,35.7178,52.0650,74.98,33,100,11.65,IR,1686142475


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_no_null[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, 'Hotel Name', '')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,avarua,CK,-21.2078,-159.7750,73,
5,port alfred,ZA,-33.5906,26.8910,38,
9,amos,CA,48.5669,-78.1162,45,
12,bredasdorp,ZA,-34.5322,20.0403,49,
15,ballina,AU,-28.8667,153.5667,70,
...,...,...,...,...,...,...
577,polyarnyy,RU,69.1989,33.4478,55,
578,gargalianoi,GR,37.0667,21.6333,74,
579,lyantor,RU,61.6195,72.1555,34,
582,damavand,IR,35.7178,52.0650,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [97]:
# Set parameters to search for a hotel

base_url = "https://api.geoapify.com/v2/places"
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20

params = {
    "filter": filters,
    "limit": limit,
    "bias": bias,
    "apiKey":geoapify_key,
    "categories": "building.accommodation"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
#    print(f"nearest hotel: {hotel_name}" )

# Display sample data
hotel_df

Starting hotel search
avarua - nearest hotel: Central Motel
port alfred - nearest hotel: No hotel found
amos - nearest hotel: Hôtel des Eskers
bredasdorp - nearest hotel: No hotel found
ballina - nearest hotel: Ballina Beach Resort
kill devil hills - nearest hotel: Mariner Days Inn & Suites
parral - nearest hotel: Camino Real
ocean city - nearest hotel: Tradewinds Motel
colchani - nearest hotel: No hotel found
benguela - nearest hotel: Hotel Mombaca
udachny - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
college - nearest hotel: Best Western Plus Chena River Lodge
selfoss - nearest hotel: Iceland Bed & Breakfast — Selfoss Hostel
stanley - nearest hotel: Hotel 52
usinsk - nearest hotel: Пилигрим
susuman - nearest hotel: No hotel found
constantia - nearest hotel: Capricorn Guesthouse
marsh harbour - nearest hotel: D's Guest House
ga

,City,Country,Lat,Lng,Humidity,Hotel Name
4,avarua,CK,-21.2078,-159.7750,73,Central Motel
5,port alfred,ZA,-33.5906,26.8910,38,No hotel found
9,amos,CA,48.5669,-78.1162,45,Hôtel des Eskers
12,bredasdorp,ZA,-34.5322,20.0403,49,No hotel found
15,ballina,AU,-28.8667,153.5667,70,Ballina Beach Resort
...,...,...,...,...,...,...
577,polyarnyy,RU,69.1989,33.4478,55,Мега
578,gargalianoi,GR,37.0667,21.6333,74,The Romanos Luxury Collection & The Westin Res...
579,lyantor,RU,61.6195,72.1555,34,Спорт
582,damavand,IR,35.7178,52.0650,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [116]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"],
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
    
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)